### Nearest Neighbor Classifier
This classifier has nothing to do with CNNs and it's very rare to use in practice, but it will make to get an idea about the basic approach to an image classification problem

##### CIFAR-10
The dataset consists of 60k tiny images that are 32 px high and wide.
Each image is labeled with one of 10 classes. These 60k images are partitioned into a training set of 50k images and test set of 10k images.
- Classes:
    - airplane
    - automobile
    - bird
    - cat
    - deer
    - dog
    - frog
    - horse
    - ship
    - truck

##### The Core Idea of kNN
- Training: memorizing all the training images and their labels. There's no complex "learning" phase
- Prediction: To classify a new test image, it compares it to every single training image to find 'k' most similar ones. It then holds a "vote" among these 'k' neighbors, and the most common label wins

-> One of the simplest possibilities is to compare the images px-by-px and add up all the difference. In other words, given to images and representing them as vectors I1,I2 a reasonable choice for comparing them might be the L1 distance:
$$
d_1(I_1, I_2) = \sum_p \left| I_1^p - I_2^p \right|
$$
 

Step-by-step implementation:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets
import pickle as pkl

# downloading CIFAR-10 training data
train_data = datasets.CIFAR10(root='./data', train=True, download=True)

# downloading CIFAR-10 test data
test_data = datasets.CIFAR10(root='./data', train=False, download=True)

# extracting the data and labels into numpy arrays
X_train, y_train = train_data.data, np.array(train_data.targets)
X_test, y_test = test_data.data, np.array(test_data.targets)

100.0%


In [2]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((50000, 32, 32, 3), (50000,), (10000, 32, 32, 3), (10000,))

the kNN classifier works with "flattened" images, meaning each 32x32x3 images should be turned into one vector of 3072 numbers:

In [9]:
X_train_flat = X_train.reshape(X_train.shape[0], 32*32*3) # reshape(..., -1) is used too if data shape is unknown
X_test_flat = X_test.reshape(X_test.shape[0], 32*32*3) # reshape(..., -1) is used too if data shape is unknown

In [10]:
X_train_flat.shape, X_test_flat.shape

((50000, 3072), (10000, 3072))

Following code is the basic implementation of Nearest Neighbor classifier using L2 distance:
$$
d_2(I_1, I_2) = \sqrt{\sum_p (I_{p1} - I_{p2})^2}
$$


In [30]:
class NearestNeighbor(object):

    def __init__(self):
        pass
    
    def train(self, X, y):
        # classifier learns / memorizes the training data
        self.Xtr = X # 
        self.ytr = y

    def predict(self, X, k=1):
        num_test = X.shape[0] # #test images to classify
        Ypred = np.zeros(num_test, dtype = self.ytr.dtype) # creating array to store predicted y values

        # looping over all test rows
        for i in range(num_test):
            # distances = np.sum(np.abs(self.Xtr - X[i, :]), axis=1) 
            # X[i, :] selects the ith test image
            
            # L2 distance for accuracy purposes
            distances = np.sqrt(np.sum(np.square(self.Xtr - X[i,:]), axis = 1))
            
            if k>1:
                closest_indices = np.argsort(distances)[:k] # indices of the 'k' mallest distances
                closest_labels = self.ytr[closest_indices]
                most_label = np.argmax(np.bincount(closest_labels))
                Ypred[i] = most_label
            # if k=1
            else:
                min_index = np.argmin(distances) # index of smallest distance 
                Ypred[i] = self.ytr[min_index] # predict the label of the nearest example

        return Ypred


In [31]:
nn = NearestNeighbor()
nn.train(X_train_flat, y_train) # training classifier on train imgs and labels
y_test_predict = nn.predict(X_test_flat[:1000], k=1) # [:1000] is for using smaller set to run fast

In [32]:
np.mean(y_test_predict == y_test[:1000])

np.float64(0.262)

'accuracy' only achieved 26.9%, which is pretty bad.
but we know that it doesn't randomly guess between classes (it would be 10% since there is 10 classes)

##### Hyperparameter tuning for Validation sets
kNN requires a set for k and for finding the best hyperparameters for model, such as the value of 'k' or the distances we used like L1 and L2 we shdn't use test set. Using test set would lead model overfitting.
- Overfitting: model performs well on train data but bad on test data
Since test set can't be used, we should split training data into two parts:
- train set
- validation set (fake test set)

In [33]:
X_val_flat = X_train_flat[:1000, :] # take first 1000 for val set
y_val = y_train[:1000]

X_train_flat = X_train_flat[1000:, :] # take last 49k for train set
y_train = y_train[1000:]

validation_accuracies = {} # to keep hyperparameters work well

nn = NearestNeighbor()
nn.train(X_train_flat, y_train)

for k in [1, 3, 5, 10, 20, 50, 100]:

    y_val_predict = nn.predict(X_val_flat, k=k)
    
    acc = np.mean(y_val_predict == y_val)

    print(f'k = {k}, accuracy: {acc*100:.2f}%')

    validation_accuracies[k] = acc # add k value and its accuracy to the array

k = 1, accuracy: 24.30%
k = 3, accuracy: 22.20%
k = 5, accuracy: 21.70%
k = 10, accuracy: 21.60%
k = 20, accuracy: 21.60%
k = 50, accuracy: 21.90%
k = 100, accuracy: 23.10%


We can now see which value of 'k' works best for model and now can stick with it

There is also another way for hyperparameter tuning called 'cross validation'.
- Cross Validation: instead of randomly picking the first 1000 datapoints to be the val set, we can get better estimate how well a certain value of k works by iterating over different validation sets and averaging the performance across these.

so far, we implemented a kNN classifier from scratch to establish a baseline and undeerstand core ML concepts like,
- Data-driven paradigm; collect data, let algo. learn patterns from examples, evaluate on test data.

kNN achieves low-accuracy simply because
- pixel-wise distances don't capture semantic similarity
- high computational cost at test time
- doesn't work well with high-D data
- that's why there are concepts like CNNs which is more complex concept

"Summary directly from cs231n lecture notes itself"
- In summary:

    - We introduced the problem of Image Classification, in which we are given a set of images that are all labeled with a single category. We are then asked to predict these categories for a novel set of test images and measure the accuracy of the predictions.
    - We introduced a simple classifier called the Nearest Neighbor classifier. We saw that there are multiple hyper-parameters (such as value of k, or the type of distance used to compare examples) that are associated with this classifier and that there was no obvious way of choosing them.
    - We saw that the correct way to set these hyperparameters is to split your training data into two: a training set and a fake test set, which we call validation set. We try different hyperparameter values and keep the values that lead to the best performance on the validation set.
    - If the lack of training data is a concern, we discussed a procedure called cross-validation, which can help reduce noise in estimating which hyperparameters work best.
    - Once the best hyperparameters are found, we fix them and perform a single evaluation on the actual test set.
    - We saw that Nearest Neighbor can get us about 30% accuracy on CIFAR-10. It is simple to implement but requires us to store the entire training set and it is expensive to evaluate on a test image.
    - Finally, we saw that the use of L1 or L2 distances on raw pixel values is not adequate since the distances correlate more strongly with backgrounds and color distributions of images than with their semantic content.